# Routing Optimization - Optimization Model

Project student team: Peter Pan; Vincent Pan; Sanjit Sokhi, Jerry Wang, Jiadi Zhang

Advisor: Amr Farahat

Creation date: 2023-03-27

This jupyter notebook utilizes Mixed Integer Programming to select orders and assign them to the drivers.
The data needed in this script must be generated from the arc generation pipeline notebook

In [50]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import datetime, timedelta
import numpy as np
from geopy.geocoders import Nominatim 
from geopy import distance 
from multidict import MultiDict
import math
import gurobipy as gp
from gurobipy import GRB
import pickle

In [79]:
optimization_time_limit = 60*60

In [123]:
with open('C:/Users/jiadiz/Desktop/PGT Trucking/initial model/Github/model_materials.pkl', 'rb') as f:
    model_materials = pickle.load(f)

In [151]:
terminals, drivers, jobs, nodes, jobs_revenues, all_arcs, flow_arcs, order_data_set, first_wave,ds = model_materials 

In [82]:
len(flow_arcs)

329084

In [83]:
def sum_for_all_driver(d,flow_arcs):
    out = []
    for ch in flow_arcs:
        if ch[0] == d:
            out.append(ch)
    return out

In [57]:
def all_driver_value(x,d,flow_arcs):
    out = []
    for ch in flow_arcs:
        
        if ch[0] == d:
            
            out.append(flow_arcs[ch])
    return out

In [58]:
# def find_the_next(path_example,k):
#     driver = k[0]
#     dest = k[2]
#     for ch in path_example:
#         if ch[0] == driver and dest == ch[1]:
#             return ch 
#     return 

In [59]:
def find_the_next(path_example,k):
    driver = k[0]
    dest = k[2]
    for ch in path_example:
        if ch[0] == driver and dest == ch[1]:
            return ch 
    return 

def generated_path(path_example):
    all_path = []
    path = ''
    for ch in path_example:
        driver = ch[0]
        if ch[1][0]!='T':
            continue 
    
        ori = ch[1]
        dest = ch[2]
        path = driver +':'+ch[1] +'->' + ch[2] 
        nex = find_the_next(path_example,ch)
        path_example.remove(ch)
        while nex:
            path = path +'->' + nex[2]
            temp = nex
            path_example.remove(nex)
            nex  = find_the_next(path_example,temp) 
            
        all_path.append(path)
        path = ''
        
    return all_path 

In [60]:
mod = gp.Model('model0')

In [61]:
y = mod.addVars(jobs, vtype=GRB.BINARY, name= "y")
x = mod.addVars(flow_arcs, vtype=GRB.BINARY, name= "x")

In [62]:
obj_fn = mod.setObjective(gp.quicksum(jobs_revenues[j]* y[j] for j in jobs), GRB.MAXIMIZE)

In [63]:
# obj_fn = mod.setObjective(gp.quicksum(y[j] for j in jobs), GRB.MAXIMIZE)

In [64]:
u = mod.addVars(nodes, vtype=GRB.INTEGER, name="u")

In [65]:
terminal_constraint = mod.addConstrs((x.sum(d, '*', t) <=1  for d in drivers for t in terminals))


In [66]:
flow_conservation = mod.addConstrs((x.sum(d, '*', j) == x.sum(d, j, '*') for d in drivers for j in nodes if j not in terminals),name="flow_conservation",
)

In [67]:
single_driver = mod.addConstrs(x.sum('*', o, d) <= 1 for (o, d) in all_arcs)

In [68]:
served_jobs = mod.addConstrs(x.sum('*', '*', j)  == y[j] for j in jobs)

In [84]:
mip_gap = 0.005  # 5% optimality gap

# mod.setParam(GRB.Param.MIPGap, mip_gap)
mod.setParam(GRB.Param.TimeLimit, optimization_time_limit)

Set parameter TimeLimit to value 3600


In [85]:
for d in drivers:
    driver =  sum_for_all_driver(d,flow_arcs)
    
    mod.addConstr(gp.quicksum(flow_arcs[ch]* x[ch] for ch in driver) <= 80)

In [86]:
mod.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700HQ CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 65340 rows, 330114 columns and 1957018 nonzeros
Model fingerprint: 0x34ea4211
Variable types: 0 continuous, 330114 integer (329584 binary)
Coefficient statistics:
  Matrix range     [4e-01, 1e+02]
  Objective range  [2e+02, 6e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]

Loaded MIP start from previous solve with objective 364716

Presolve removed 45872 rows and 117932 columns (presolve time = 5s) ...
Presolve removed 49812 rows and 118049 columns (presolve time = 10s) ...
Presolve removed 49812 rows and 118049 columns
Presolve time: 10.50s
Presolved: 15528 rows, 212065 columns, 876319 nonzeros
Variable types: 0 continuous, 212065 integer (212065 binary)
Deterministic concurrent LP optimizer: primal simplex, dual simplex

     0     0 381418.814    0  587 367696.349 381418.814  3.73%     -  273s
     0     0 381418.814    0  587 367696.349 381418.814  3.73%     -  275s
H    0     2                    369043.62072 380519.966  3.11%     -  319s
     0     2 380519.966    0  587 369043.621 380519.966  3.11%     -  319s
     1     4 380519.966    1  593 369043.621 380519.966  3.11%   439  332s
     3     8 380519.966    2  581 369043.621 380519.966  3.11%  1024  348s
     7    12 380519.966    3  564 369043.621 380519.966  3.11%   999  359s
    11    16 380519.966    3  573 369043.621 380519.966  3.11%   917  376s
    15    20 380519.966    4  555 369043.621 380519.966  3.11%   964  383s
    19    24 380519.966    5  550 369043.621 380519.966  3.11%   872  394s
    23    28 380519.966    6  577 369043.621 380519.966  3.11%   824  403s
    27    32 380519.966    7  585 369043.621 380519.966  3.11%   769  409s
    31    35 380519.966    8  580 369043.621 380519.966  3.11%   712  419s
    36    39 380519.966  

In [98]:
complete_path = []
for i in flow_arcs:
    if (x[i].x >=0.99):
        complete_path.append(i)

In [99]:
complete_job = complete_path.copy()
final_result = generated_path(complete_job)
while complete_job != []:
    final_result+= generated_path(complete_job)
final_result

['Driver457:T11->7043->4586->3519->8013->T11',
 'Driver130:T26->6132->2248->3528->2233->T26',
 'Driver228:T20->9730->5071->1204->39->2414->T20',
 'Driver181:T12->5406->6433->7061->T12',
 'Driver378:T4->9659->1540->188->4960->T4',
 'Driver453:T14->1355->210->584->7849->T14',
 'Driver331:T29->6099->9190->9818->T29',
 'Driver477:T19->3677->4392->2939->T19',
 'Driver273:T23->3925->4775->5650->2103->T23',
 'Driver91:T22->4711->8121->7848->7259->T22',
 'Driver361:T16->5359->5818->9556->4204->T16',
 'Driver109:T21->9665->4929->3144->9477->T21',
 'Driver344:T25->4173->4857->2415->8808->T25',
 'Driver247:T5->8431->2895->4925->T5',
 'Driver82:T9->3208->4533->T9',
 'Driver458:T3->7607->2636->4065->T3',
 'Driver165:T28->1362->3372->9067->T28',
 'Driver223:T1->552->9361->4958->T1',
 'Driver12:T18->7312->6329->4340->T18',
 'Driver210:T27->5231->9442->8198->T27',
 'Driver39:T10->1952->6289->8039->6568->T10',
 'Driver294:T13->9836->8777->423->5333->T13',
 'Driver550:T8->495->5578->1638->T8',
 'Driver2

In [101]:
selected_jobs_count = 0
for i in jobs:
    #if (x[i].x >= 0.99):
        #print(x[i].x)
        if y[i].x > 0.5:
            selected_jobs_count += 1 

In [102]:
print('selected_jobs_count :' +str(selected_jobs_count), 'job_count_in_job_pool :' + str(len(order_data_set)), 'job_selection_rate :' + str(selected_jobs_count/len(order_data_set)))

selected_jobs_count :102 job_count_in_job_pool :500 job_selection_rate :0.204


In [103]:
print('optimized_revenue :' +str(mod.ObjVal), 'revenue_in_pool:' + str(order_data_set['LineHaulRevenue'].sum()), 'optimized_revenue_to_pool_revenue_rate :' + str(mod.ObjVal/order_data_set['LineHaulRevenue'].sum()))

optimized_revenue :373270.93893985846 revenue_in_pool:1511384.0890695387 optimized_revenue_to_pool_revenue_rate :0.24697291816116523


In [166]:
routes = final_result

In [167]:
dashboard_materials = final_result, first_wave, ds

In [170]:
with open('create_dashboard.pkl', 'wb') as f:
    pickle.dump(dashboard_materials, f)

In [171]:
routes

['Driver457:T11->7043->4586->3519->8013->T11',
 'Driver130:T26->6132->2248->3528->2233->T26',
 'Driver228:T20->9730->5071->1204->39->2414->T20',
 'Driver181:T12->5406->6433->7061->T12',
 'Driver378:T4->9659->1540->188->4960->T4',
 'Driver453:T14->1355->210->584->7849->T14',
 'Driver331:T29->6099->9190->9818->T29',
 'Driver477:T19->3677->4392->2939->T19',
 'Driver273:T23->3925->4775->5650->2103->T23',
 'Driver91:T22->4711->8121->7848->7259->T22',
 'Driver361:T16->5359->5818->9556->4204->T16',
 'Driver109:T21->9665->4929->3144->9477->T21',
 'Driver344:T25->4173->4857->2415->8808->T25',
 'Driver247:T5->8431->2895->4925->T5',
 'Driver82:T9->3208->4533->T9',
 'Driver458:T3->7607->2636->4065->T3',
 'Driver165:T28->1362->3372->9067->T28',
 'Driver223:T1->552->9361->4958->T1',
 'Driver12:T18->7312->6329->4340->T18',
 'Driver210:T27->5231->9442->8198->T27',
 'Driver39:T10->1952->6289->8039->6568->T10',
 'Driver294:T13->9836->8777->423->5333->T13',
 'Driver550:T8->495->5578->1638->T8',
 'Driver2